In [3]:
import os, sys
module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)

%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from skimage import io

# out = widgets.Output()

from lib.labelling import *


In [ ]:
obj.mask_dir

In [ ]:
'images/images/images/'.rsplit('images/', 1)

In [ ]:
from os import path